In [50]:
#Keras support
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121 as DenseNet
from keras.optimizers import SGD
from keras.layers import Input, Dense
from keras.models import Model

from pandas import read_csv

from common import constants

In [43]:
label_df = read_csv(constants.PROCESSED_DATASET_MAPPINGS['labels'])
n_classes = len(set(label_df["Id"]))

print("Number of classes: {}".format(n_classes))

Number of classes: 5005


In [53]:
input_shape = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
                    rescale=1./255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    validation_split = 0.2)

train_gen = datagen.flow_from_dataframe(
                    label_df,
                    constants.PROCESSED_DATASET_MAPPINGS['train'],
                    x_col = 'Image',
                    y_col = 'Id',
                    target_size=input_shape,
                    batch_size=batch_size,
                    class_mode='categorical',
                    subset = 'training')

val_gen = datagen.flow_from_dataframe(
                    label_df,
                    constants.PROCESSED_DATASET_MAPPINGS['train'],
                    x_col = 'Image',
                    y_col = 'Id',
                    target_size=input_shape,
                    batch_size=batch_size,
                    class_mode='categorical',
                    subset='validation')

Found 820 images belonging to 5005 classes.
Found 204 images belonging to 5005 classes.


In [51]:
inputs = Input(shape = (224, 224, 3))
X = DenseNet(weights = 'imagenet')(inputs)
X = Dense(n_classes, activation = 'softmax')(X)

model = Model(inputs = [inputs], outputs = [X])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
densenet121 (Model)          (None, 1000)              8062504   
_________________________________________________________________
dense_1 (Dense)              (None, 5005)              5010005   
Total params: 13,072,509
Trainable params: 12,988,861
Non-trainable params: 83,648
_________________________________________________________________


In [54]:
model.fit_generator(
    train_gen,
    steps_per_epoch=630,
    validation_data=val_gen,
    validation_steps = 150,
    epochs=50)

Epoch 1/50
  3/630 [..............................] - ETA: 56:09:10 - loss: 8.5182 - acc: 0.0000e+0 - ETA: 39:39:15 - loss: 8.5176 - acc: 0.0156    - ETA: 36:03:12 - loss: 8.5161 - acc: 0.0938

KeyboardInterrupt: 